In [3]:
import pandas as pd

# Path to the downloaded file in Kaggle environment
file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
print("Before date conversion:")
print(data.head())

# Convert the 'Datetime' column to datetime
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Filter the data to keep only rows from the year 2010
data = data[data['Datetime'].dt.year >= 2010]

# Format the 'Datetime' column to the desired format: %Y-%m-%d %H:%M
data['Datetime'] = data['Datetime'].dt.strftime('%Y-%m-%d %H:%M')

# Display the first few rows after filtering and conversion
print("After filtering for >=2010 and date conversion:")
print(data.head())

# Save the modified DataFrame back to a new CSV file
output_file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv"
data.to_csv(output_file_path, index=False)

# Confirm that the file has been saved
print(f"Modified file saved as {output_file_path}")

Before date conversion:
              Datetime  ABI   ACT  AMA  ALB  JFK  LGA
0  1949-01-01 00:00:00  6.1  10.6  2.2 -3.3 -3.3 -3.9
1  1949-01-01 01:00:00  3.3   6.7  1.1 -3.3 -3.3 -3.9
2  1949-01-01 02:00:00  2.2   5.6  0.6 -3.3 -3.3 -3.9
3  1949-01-01 03:00:00  1.7   4.4  0.0 -2.8 -3.3 -3.9
4  1949-01-01 04:00:00  1.7   3.9  1.1 -2.8 -3.3 -3.9
After filtering for >=2010 and date conversion:
                Datetime  ABI  ACT  AMA  ALB  JFK  LGA
470398  2010-01-01 00:00  1.1  6.7 -0.6 -1.7  1.1  1.1
470399  2010-01-01 01:00  1.1  6.1 -3.3 -1.7  1.1  1.1
470400  2010-01-01 02:00  1.7  5.6 -3.9 -1.7  0.6  1.1
470401  2010-01-01 03:00  1.1  5.0 -3.9 -1.7  0.6  1.1
470402  2010-01-01 04:00  1.1  4.4 -5.0 -1.1  0.6  0.6
Modified file saved as S:\spatiotemporal-analysis\spacetimeformer-main\spacetimeformer\data\temperature-v1_modified.csv


!python S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\train.py spacetimeformer asos --context_points 128 --target_points 40 --run_name spacetimeformer_asos --batch_size 32 --warmup_steps 1000 --d_model 200 --d_ff 700 --enc_layers 1 --dec_layers 1 --dropout_emb .1 --dropout_ff .3 --d_qk 30 --d_v 30 --n_heads 10 --patience 10 --decay_factor .8 --data_path S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv

In [1]:
import pandas as pd

def subset_csv(src_path: str, dest_path: str, columns: list):
    """
    Reads a CSV file from src_path, extracts the specified columns, and writes the subset to dest_path.
    
    Parameters:
        src_path (str): Path to the source CSV file.
        dest_path (str): Path to save the subset CSV file.
        columns (list): List of column names to extract from the source CSV.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(src_path)
    
    # Create a subset of the DataFrame with the specified columns
    subset = df[columns]
    
    # Save the subset to the destination CSV file
    subset.to_csv(dest_path, index=False)
    print(f"Subset saved to {dest_path}")

In [4]:
subset_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_merged.csv', '/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv', ['SETTLEMENTDATE', "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_VIC", "RRP_VIC",])
subset_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_merged.csv', '/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv', ['SETTLEMENTDATE', "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD",])

Subset saved to /Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv
Subset saved to /Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv


In [5]:
pd.read_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv')

,SETTLEMENTDATE,TOTALDEMAND_NSW,RRP_NSW,TOTALDEMAND_VIC,RRP_VIC
0,2015-02-01 00:00,6529.375000,19.210000,4485.130000,14.590000
1,2015-02-01 01:00,6145.710000,18.195000,4055.830000,12.925000
2,2015-02-01 02:00,5782.555000,16.450000,3701.360000,10.440000
3,2015-02-01 03:00,5600.055000,14.200000,3488.495000,6.660000
4,2015-02-01 04:00,5619.520000,15.955000,3455.595000,4.380000
...,...,...,...,...,...
88100,2025-02-18 20:00,7977.682500,106.371667,5107.343333,93.497500
88101,2025-02-18 21:00,7653.483333,100.095833,4648.734167,87.070833
88102,2025-02-18 22:00,7361.791667,97.980000,4341.420833,84.615000
88103,2025-02-18 23:00,7002.186667,96.224167,4404.169167,86.157500


In [6]:
pd.read_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv')

,SETTLEMENTDATE,TOTALDEMAND_NSW,RRP_NSW,TOTALDEMAND_QLD,RRP_QLD
0,2015-02-01 00:00,6529.375000,19.210000,5687.755000,17.855000
1,2015-02-01 01:00,6145.710000,18.195000,5349.120000,16.100000
2,2015-02-01 02:00,5782.555000,16.450000,5148.620000,14.420000
3,2015-02-01 03:00,5600.055000,14.200000,5045.510000,13.025000
4,2015-02-01 04:00,5619.520000,15.955000,5066.075000,14.620000
...,...,...,...,...,...
88100,2025-02-18 20:00,7977.682500,106.371667,8026.143333,112.365000
88101,2025-02-18 21:00,7653.483333,100.095833,7663.199167,102.720833
88102,2025-02-18 22:00,7361.791667,97.980000,7187.470833,102.015000
88103,2025-02-18 23:00,7002.186667,96.224167,6736.655000,100.227500


In [2]:
import pandas as pd

def filter_columns_and_save(input_file_path, selected_columns, output_file_path):
    # Read the original dataframe from the input CSV file
    df = pd.read_csv(input_file_path)
    
    # Filter the dataframe to only include the selected columns
    df_filtered = df[selected_columns]
    
    # Save the filtered dataframe to the specified output file path
    df_filtered.to_csv(output_file_path, index=False)
    print(f"Filtered dataframe saved to {output_file_path}")

In [3]:
l = [['NSW'],
 ['TAS'],
 ['QLD'],
 ['VIC'],
 ['SA'],
 ['NSW', 'QLD'],
 ['NSW', 'VIC'],
 ['QLD', 'SA'],
 ['TAS', 'VIC'],
 ['NSW', 'SA', 'VIC'],
 ['NSW', 'QLD', 'VIC'],
 ['NSW', 'QLD', 'SA'],
 ['NSW', 'TAS', 'VIC'],
 ['NSW', 'QLD', 'SA', 'VIC'],
 ['NSW', 'SA', 'TAS', 'VIC'],
 ['NSW', 'QLD', 'SA', 'TAS', 'VIC']]

for i in l:
    input_file_path = 'C:\\Users\\vm-user\\Downloads\\aemo_merged.csv'
    selected_columns = ['SETTLEMENTDATE']
    for j in i:
        selected_columns.append(f'TOTALDEMAND_{j}')
        selected_columns.append(f'RRP_{j}')
    output_file_path = f'C:\\Users\\vm-user\\Downloads\\{"_".join(i)}.csv'
    filter_columns_and_save(input_file_path, selected_columns, output_file_path)

Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\TAS.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\QLD.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\SA.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\QLD_SA.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\TAS_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_SA_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD_SA.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_TAS_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD_SA_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_SA_T

In [3]:
filter_columns_and_save(
    "C:\\Users\\vm-user\\Downloads\\aemo_merged.csv", 
    ["SETTLEMENTDATE", "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD", "TOTALDEMAND_VIC", "RRP_VIC"], 
    "C:\\Users\\vm-user\\Downloads\\aemo_nsw_qld_vic.csv"
)

Filtered dataframe saved to C:\Users\vm-user\Downloads\aemo_nsw_qld_vic.csv


In [4]:
filter_columns_and_save(
    "C:\\Users\\vm-user\\Downloads\\aemo_merged.csv", 
    ["SETTLEMENTDATE", "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD", "TOTALDEMAND_VIC", "RRP_VIC", "TOTALDEMAND_TAS", "RRP_TAS"], 
    "C:\\Users\\vm-user\\Downloads\\aemo_nsw_qld_vic_tas.csv"
)

Filtered dataframe saved to C:\Users\vm-user\Downloads\aemo_nsw_qld_vic_tas.csv


# EU

In [115]:
all_countries = pd.read_csv("C:\\Users\\vm-user\\Downloads\\all_countries.csv")

In [116]:
all_countries.head()

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe)
0,Austria,AUT,2015-01-01 00:00:00,2015-01-01 01:00:00,22.34
1,Belgium,BEL,2015-01-01 00:00:00,2015-01-01 01:00:00,36.56
2,Czechia,CZE,2015-01-01 00:00:00,2015-01-01 01:00:00,24.20
3,Denmark,DNK,2015-01-01 00:00:00,2015-01-01 01:00:00,18.29
4,Estonia,EST,2015-01-01 00:00:00,2015-01-01 02:00:00,23.37


In [117]:
countries_map = {
    'Austria': 'AT',
    'Belgium': 'BE',
    'Bulgaria': 'BG',
    'Croatia': 'HR',
    'Czechia': 'CZ',
    'Denmark': 'DK',
    'Estonia': 'EE',
    'Finland': 'FI',
    'France': 'FR',
    'Germany': 'DE',
    'Greece': 'GR',
    'Hungary': 'HU',
    'Ireland': 'IE',
    'Italy': 'IT',
    'Latvia': 'LV',
    'Lithuania': 'LT',
    'Luxembourg': 'LU',
    'Montenegro': 'ME',
    'Netherlands': 'NL',
    'North Macedonia': 'MK',
    'Norway': 'NO',
    'Poland': 'PL',
    'Portugal': 'PT',
    'Romania': 'RO',
    'Serbia': 'RS',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Spain': 'ES',
    'Sweden': 'SE',
    'Switzerland': 'CH',
    'United Kingdom': 'GB'
}

In [118]:
len(set(countries_map.values())), len(countries_map.values())

(31, 31)

In [119]:
# Replace the Country column values with their two-letter codes
all_countries['Country'] = all_countries['Country'].map(countries_map)

# Optionally, check the result
print("Unique two-letter country codes in Country column:", all_countries['Country'].unique())

Unique two-letter country codes in Country column: ['AT' 'BE' 'CZ' 'DK' 'EE' 'FI' 'FR' 'DE' 'GR' 'HU' 'IT' 'LV' 'LT' 'LU'
 'NL' 'NO' 'PL' 'PT' 'RO' 'SK' 'SI' 'ES' 'SE' 'CH' 'GB' 'BG' 'RS' 'HR'
 'ME' 'MK' 'IE']


In [120]:
len(set(countries_map.values())), len(countries_map.values())

(31, 31)

In [121]:
import pandas as pd

def pivot_and_resample(df):
    """
    Transforms the dataset by pivoting countries into columns and resampling the time series.

    Parameters:
    df (pd.DataFrame): Input dataframe containing country-wise electricity prices.

    Returns:
    pd.DataFrame: Resampled dataframe with hourly prices for each country as separate columns.
    """
    # Convert 'Datetime (UTC)' to datetime format
    df['Datetime (UTC)'] = pd.to_datetime(df['Datetime (UTC)'])

    # Drop 'Datetime (Local)' as it's not needed
    df = df.drop(columns=['Datetime (Local)'])

    # Rename 'Price (EUR/MWhe)' to 'Price'
    df = df.rename(columns={'Price (EUR/MWhe)': 'Price'})

    # Pivot the table to make countries as columns
    df_pivoted = df.pivot_table(index='Datetime (UTC)', columns='Country', values='Price', aggfunc='mean')

    # Resample to ensure hourly intervals from min to max timestamp
    df_resampled = df_pivoted.resample('h').mean()

    # Reset index to bring datetime back as a column
    df_resampled.reset_index(inplace=True)

    return df_resampled

# Example usage
final_df = pivot_and_resample(all_countries)

In [122]:
df = final_df.copy()
for col in df.columns[1:]:
    non_null_series = df[col].dropna()  # Remove NaN values to find the first non-null value
    if not non_null_series.empty:
        first_index = non_null_series.index[0]  # Find the first non-null value's index
        first_value = non_null_series.iloc[0]  # Get the first non-null value
        
        # Replace consecutive occurrences with NaN, starting from first_index
        for i in range(first_index + 1, len(df)):
            if pd.isna(df[col].iloc[i]) or df[col].iloc[i] != first_value:
                break  # Stop if a different value or NaN appears
            df.loc[i-1, col] = float("nan")  # Set previous consecutive occurrences to NaN

# Remove initial rows where all country columns are NaN (excluding 'Datetime')
df_cleaned = df.dropna(subset=df.columns[1:], how='all').reset_index(drop=True)

In [123]:
def has_1hr_intervals(time_series):
    time_series = pd.to_datetime(time_series).sort_values()
    expected_interval = pd.Timedelta(hours=1)
    return (time_series.diff().dropna() == expected_interval).all()

In [124]:
has_1hr_intervals(df_cleaned['Datetime (UTC)'])

True

In [125]:
df_cleaned

Country,Datetime (UTC),AT,BE,BG,CH,CZ,DE,DK,EE,ES,...,MK,NL,NO,PL,PT,RO,RS,SE,SI,SK
0,2015-01-01 00:00:00,NaN,NaN,NaN,43.43,24.20,NaN,18.29,23.37,48.10,...,NaN,NaN,27.35,NaN,48.10,NaN,NaN,23.37,23.25,24.20
1,2015-01-01 01:00:00,NaN,NaN,NaN,38.08,22.06,NaN,16.04,19.33,47.33,...,NaN,NaN,27.23,NaN,47.33,NaN,NaN,19.33,22.20,22.06
2,2015-01-01 02:00:00,NaN,NaN,NaN,35.47,20.27,NaN,14.60,17.66,42.27,...,NaN,NaN,27.15,NaN,42.27,NaN,NaN,17.66,19.56,20.27
3,2015-01-01 03:00:00,NaN,NaN,NaN,30.83,19.17,NaN,14.95,17.53,38.41,...,NaN,NaN,27.14,NaN,38.41,NaN,NaN,17.53,18.88,19.17
4,2015-01-01 04:00:00,NaN,NaN,NaN,28.26,17.90,NaN,14.50,18.07,35.72,...,NaN,NaN,27.29,NaN,35.72,NaN,NaN,18.07,18.39,17.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88818,2025-02-17 18:00:00,214.41,215.46,224.35,203.38,216.50,214.10,215.38,219.35,215.52,...,148.2,215.43,94.09,218.20,215.52,216.86,223.78,157.34,216.46,217.09
88819,2025-02-17 19:00:00,178.47,177.95,224.35,171.35,179.08,181.78,177.87,180.91,200.00,...,148.2,177.93,88.03,180.90,200.00,179.47,195.21,130.24,179.03,179.71
88820,2025-02-17 20:00:00,159.23,155.38,224.35,159.98,151.40,161.50,155.19,155.95,155.62,...,148.2,155.30,82.59,154.09,155.62,165.32,174.31,110.05,161.59,172.18
88821,2025-02-17 21:00:00,159.23,139.51,224.35,159.98,151.40,161.50,155.19,155.95,140.00,...,148.2,139.39,82.59,154.09,155.62,161.88,174.31,110.05,161.59,172.18


In [126]:
import pandas as pd

def filter_and_clean_dataframe(df, selected_columns):
    """
    Filters the given DataFrame to retain only the selected columns, 
    and removes initial and bottom-most rows where all selected columns are NaN.
    
    Parameters:
        df (pd.DataFrame): The original DataFrame.
        selected_columns (list): List of column names to retain.
    
    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Ensure 'Datetime (UTC)' is included in the filtered columns
    filtered_columns = ['Datetime (UTC)'] + selected_columns if 'Datetime (UTC)' in df.columns else selected_columns
    df_filtered = df[filtered_columns]

    # Drop initial rows where all selected columns (excluding 'Datetime') are NaN
    start_index = df_filtered[selected_columns].first_valid_index()
    df_cleaned = df_filtered.loc[start_index:].reset_index(drop=True)
    
    # Drop bottom-most rows where all selected columns (excluding 'Datetime') are NaN
    end_index = df_cleaned[selected_columns].last_valid_index()
    df_cleaned = df_cleaned.loc[:end_index].reset_index(drop=True)
    df_cleaned.fillna(10**6, inplace=True)  # Replace NaNs with a large number

    return df_cleaned

In [127]:
from geopy.geocoders import Nominatim
import requests
import numpy as np

# List of countries
countries = ['Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland',
'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia',
'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland',
'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
'Switzerland', 'United Kingdom', 'Bulgaria', 'Serbia', 'Croatia',
'Montenegro', 'North Macedonia', 'Ireland']

lat = [47.516231, 50.503887, 49.817492, 56.26392, 58.595272, 61.92411,
       46.227638, 51.165691, 39.074208, 47.162494, 41.87194, 56.879635,
       55.169438, 49.815273, 52.132633, 60.472024, 51.919438, 39.399872,
       45.943161, 48.669026, 46.151241, 40.463667, 60.128161, 46.818188,
       55.378051, 42.733883, 44.016521, 45.1, 42.708678, 41.608635, 53.41291]

long = [14.550072, 4.469936, 15.472962, 9.501785, 25.013607, 25.748151,
        2.213749, 10.451526, 21.824312, 19.503304, 12.56738, 24.603189,
        23.881275, 6.129583, 5.291266, 8.468946, 19.145136, -8.224454,
        24.96676, 19.699024, 14.995463, -3.74922, 18.643501, 8.227512,
        -3.435973, 25.48583, 21.005859, 15.2, 19.37439, 21.745275, -8.24389]

area_km_sq = [82520, 30280, 77187, 42434, 42388, 303815, 
       640427, 349390, 128900, 91260, 295717, 62230,
       62610, 2574, 33893, 366704, 304255,
       91606, 230080, 48080, 20151, 498980, 407284,
       39510, 242741, 108489, 88499, 55974,
       13452, 25220, 68883]

eu_data = {
    'countries': countries,
    'lat': lat,
    'long': long,
    'area_km_sq': area_km_sq
}


def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Function to calculate the effective radius based on area
def effective_radius(area):
    return np.sqrt(area / np.pi)

# Prepare the data into a DataFrame
eu_df = pd.DataFrame(eu_data)

# Calculate the effective radius for each country
eu_df['effective_radius'] = eu_df['area_km_sq'].apply(effective_radius)

# Initialize list to store both normalized and physical distances
distances = []

# Loop through pairs of countries to calculate distances
for i in range(len(eu_df)):
    for j in range(len(eu_df)):
        # Get the coordinates, areas and radii
        lat1, lon1, r1 = eu_df.iloc[i]['lat'], eu_df.iloc[i]['long'], eu_df.iloc[i]['effective_radius']
        lat2, lon2, r2 = eu_df.iloc[j]['lat'], eu_df.iloc[j]['long'], eu_df.iloc[j]['effective_radius']
        
        # Calculate the physical distance between the centroids using the Haversine formula
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Calculate the normalized distance
        D_norm = distance / (r1 + r2)
        
        # Append the results to the list
        distances.append({
            'Country 1': eu_df.iloc[i]['countries'],
            'Country 2': eu_df.iloc[j]['countries'],
            'Physical Distance (km)': distance,
            'Normalized Distance': D_norm
        })

distances_df = pd.DataFrame(distances)

In [128]:
distances_df

,Country 1,Country 2,Physical Distance (km),Normalized Distance
0,Austria,Austria,0.000000,0.000000
1,Austria,Belgium,805.904977,3.096702
2,Austria,Czechia,264.705969,0.830276
3,Austria,Denmark,1031.865905,3.707866
4,Austria,Estonia,1412.866158,5.078086
...,...,...,...,...
956,Ireland,Serbia,2364.256758,7.483855
957,Ireland,Croatia,1923.646715,6.832214
958,Ireland,Montenegro,2350.806279,11.010233
959,Ireland,North Macedonia,2578.886347,10.850582


In [129]:
import pandas as pd
import math

def closest_locations(df, location_col, other_location_col, distance_col, n):
    # Create an empty dictionary to store the results
    result = {}
    
    # Get unique values in the location column
    unique_locations = df[location_col].unique()
    
    for location in unique_locations:
        # Filter the dataframe for rows corresponding to the current location
        location_df = df[df[location_col] == location]
        
        # Sort the rows based on the distance column
        location_df = location_df.sort_values(by=distance_col)
        
        # Create subsets that progressively add the next 'n' closest locations
        closest_n_locations = []
            
        num_steps = math.ceil(len(location_df) / n)
        for i in range(1, num_steps + 1):
            # Ensure we don't go past the length of the DataFrame
            end_index = min(i * n, len(location_df))
            closest_n_locations.append(location_df.iloc[:end_index][other_location_col].tolist())        # Add the result for the current location to the dictionary
            result[location] = closest_n_locations
    
    return result

In [130]:
len(distances_df[distances_df['Country 1'] == 'Austria'].sort_values(by='Normalized Distance'))

31

In [131]:
def unique_subsets(closest_dict):
    # Set to hold unique subsets (as sorted tuples)
    unique_sets = set()

    # Iterate over each location's list of subsets
    for location, subsets in closest_dict.items():
        for subset in subsets:
            # Convert each subset to a sorted tuple (for order-insensitivity)
            unique_sets.add(tuple(sorted(subset)))
    
    # Convert the set of unique tuples back to a list of lists
    unique_subsets_list = [list(subset) for subset in unique_sets]
    
    # Sort the subsets by length
    unique_subsets_list.sort(key=len)
    
    return unique_subsets_list

In [132]:
country_subsets = unique_subsets(closest_locations(distances_df, 'Country 1', 'Country 2', 'Normalized Distance', 10))

In [133]:
len(country_subsets)

58

In [134]:
df_cleaned['Datetime (UTC)'] = pd.to_datetime(df_cleaned['Datetime (UTC)'])
df_cleaned['Datetime (UTC)'] = df_cleaned['Datetime (UTC)'].dt.strftime('%Y-%m-%d %H:%M')

for country in countries_map:
    df_to_save = filter_and_clean_dataframe(df_cleaned, [countries_map[country]])
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\eu_{countries_map[country]}.csv"
    # Print a message to see if the df has null values
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {countries_map[country]} contains null values.")
    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

for subset in country_subsets:
    subset = [countries_map[country] for country in subset]
    df_to_save = filter_and_clean_dataframe(df_cleaned, subset)
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {countries_map[country]} contains null values.")
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\eu_{'_'.join(subset)}.csv"
    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

Saved: C:\Users\vm-user\Downloads\eu_AT.csv
Saved: C:\Users\vm-user\Downloads\eu_BE.csv
Saved: C:\Users\vm-user\Downloads\eu_BG.csv
Saved: C:\Users\vm-user\Downloads\eu_HR.csv
Saved: C:\Users\vm-user\Downloads\eu_CZ.csv
Saved: C:\Users\vm-user\Downloads\eu_DK.csv
Saved: C:\Users\vm-user\Downloads\eu_EE.csv
Saved: C:\Users\vm-user\Downloads\eu_FI.csv
Saved: C:\Users\vm-user\Downloads\eu_FR.csv
Saved: C:\Users\vm-user\Downloads\eu_DE.csv
Saved: C:\Users\vm-user\Downloads\eu_GR.csv
Saved: C:\Users\vm-user\Downloads\eu_HU.csv
Saved: C:\Users\vm-user\Downloads\eu_IE.csv
Saved: C:\Users\vm-user\Downloads\eu_IT.csv
Saved: C:\Users\vm-user\Downloads\eu_LV.csv
Saved: C:\Users\vm-user\Downloads\eu_LT.csv
Saved: C:\Users\vm-user\Downloads\eu_LU.csv
Saved: C:\Users\vm-user\Downloads\eu_ME.csv
Saved: C:\Users\vm-user\Downloads\eu_NL.csv
Saved: C:\Users\vm-user\Downloads\eu_MK.csv
Saved: C:\Users\vm-user\Downloads\eu_NO.csv
Saved: C:\Users\vm-user\Downloads\eu_PL.csv
Saved: C:\Users\vm-user\Download

In [ ]:
df_cleaned.isna().sum()

In [59]:
len(set(country_subsets[-1]) & set(country_subsets[-2])), len(set(country_subsets[-1]) | set(country_subsets[-2]))

(29, 31)

In [60]:
len(set(country_subsets[-1]))

30

# NYISO

In [153]:
nyiso_data_df = pd.read_csv("C:\\Users\\vm-user\\Downloads\\nyiso.csv")

In [154]:
from geopy.geocoders import Nominatim
import requests

# List of countries
regions = ['CAPITL', 'CENTRL', 'MHK VL', 'HUD VL', 'LONGIL', 'N.Y.C.', 'WEST', 'GENESE', 'NORTH', 'DUNWOD', 'MILLWD']

long = [-73.9820406, -76.7300675, -75.1443401, -74.0753086, -73.0188755, -73.9247654, -78.6001935, -77.5107558, -74.0169224, -73.7846149, -73.8141997]

lat = [43.1823555, 42.6094099, 43.4161459, 41.7112736, 40.8443418, 40.6944843, 42.6613719, 42.8694038, 44.5393904, 41.0350397, 41.2394963]

area_km_sq = [41206.36, 39600.998, 74038.96, 18453.5, 5795.6, 1566.34, 29527.06, 9535.63, 15199.63, 905.66, 743.044]

nyiso_data = {
    'regions': regions,
    'lat': lat,
    'long': long,
    'area_km_sq': area_km_sq
}


def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Function to calculate the effective radius based on area
def effective_radius(area):
    return np.sqrt(area / np.pi)

# Prepare the data into a DataFrame
nyiso_df = pd.DataFrame(nyiso_data)

# Calculate the effective radius for each country
nyiso_df['effective_radius'] = nyiso_df['area_km_sq'].apply(effective_radius)

# Initialize list to store both normalized and physical distances
distances = []

# Loop through pairs of countries to calculate distances
for i in range(len(nyiso_df)):
    for j in range(len(nyiso_df)):
        # Get the coordinates, areas and radii
        lat1, lon1, r1 = nyiso_df.iloc[i]['lat'], nyiso_df.iloc[i]['long'], nyiso_df.iloc[i]['effective_radius']
        lat2, lon2, r2 = nyiso_df.iloc[j]['lat'], nyiso_df.iloc[j]['long'], nyiso_df.iloc[j]['effective_radius']
        
        # Calculate the physical distance between the centroids using the Haversine formula
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Calculate the normalized distance
        D_norm = distance / (r1 + r2)
        
        # Append the results to the list
        distances.append({
            'Region 1': nyiso_df.iloc[i]['regions'],
            'Region 2': nyiso_df.iloc[j]['regions'],
            'Physical Distance (km)': distance,
            'Normalized Distance': D_norm
        })

distances_df = pd.DataFrame(distances)

In [155]:
len(unique_subsets(closest_locations(distances_df, 'Region 1', 'Region 2', 'Normalized Distance', 3)))

19

In [156]:
region_subsets = unique_subsets(closest_locations(distances_df, 'Region 1', 'Region 2', 'Normalized Distance', 3))

In [157]:
len(region_subsets)

19

In [159]:
region_subsets = sorted(region_subsets, key=len)
region_subsets

[['CENTRL', 'GENESE', 'WEST'],
 ['DUNWOD', 'HUD VL', 'N.Y.C.'],
 ['DUNWOD', 'HUD VL', 'LONGIL'],
 ['DUNWOD', 'HUD VL', 'MILLWD'],
 ['CAPITL', 'MHK VL', 'NORTH'],
 ['CENTRL', 'GENESE', 'MHK VL'],
 ['CAPITL', 'CENTRL', 'MHK VL'],
 ['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'NORTH'],
 ['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'NORTH', 'WEST'],
 ['CAPITL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD'],
 ['DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.'],
 ['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'WEST'],
 ['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH'],
 ['CAPITL',
  'CENTRL',
  'DUNWOD',
  'HUD VL',
  'LONGIL',
  'MHK VL',
  'MILLWD',
  'N.Y.C.',
  'NORTH'],
 ['CAPITL',
  'CENTRL',
  'DUNWOD',
  'GENESE',
  'HUD VL',
  'MHK VL',
  'MILLWD',
  'NORTH',
  'WEST'],
 ['CAPITL',
  'CENTRL',
  'GENESE',
  'HUD VL',
  'LONGIL',
  'MHK VL',
  'MILLWD',
  'NORTH',
  'WEST'],
 ['CAPITL',
  'CENTRL',
  'DUNWOD',
  'HUD VL',
  'LONGIL',
  'MHK VL',
  'MILLWD

In [160]:
nyiso_data_df.isna().sum()

Time Stamp    0
CAPITL        0
CENTRL        0
DUNWOD        0
GENESE        0
HUD VL        0
LONGIL        0
MHK VL        0
MILLWD        0
N.Y.C.        0
NORTH         0
WEST          0
dtype: int64

In [161]:
import pandas as pd

def filter_nyiso(df, selected_columns):
    """
    Filters the given DataFrame to retain only the selected columns, 
    and removes initial and bottom-most rows where all selected columns are NaN.
    
    Parameters:
        df (pd.DataFrame): The original DataFrame.
        selected_columns (list): List of column names to retain.
    
    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Ensure 'Datetime (UTC)' is included in the filtered columns
    filtered_columns = ['Time Stamp'] + selected_columns if 'Time Stamp' in df.columns else selected_columns
    df_filtered = df[filtered_columns]
    return df_filtered

In [162]:
nyiso_data_df['Time Stamp'] = pd.to_datetime(nyiso_data_df['Time Stamp'])
nyiso_data_df['Time Stamp'] = nyiso_data_df['Time Stamp'].dt.strftime('%Y-%m-%d %H:%M')

for region in regions:
    df_to_save = filter_nyiso(nyiso_data_df, [region])
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\nyiso_{region}.csv"
    # Print a message to see if the df has null values
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {region} contains null values.")

    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

for subset in region_subsets:
    df_to_save = filter_nyiso(nyiso_data_df, subset)
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\nyiso_{'_'.join(subset)}.csv"
    # Print a message to see if the df has null values
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {subset} contains null values.")
    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

Saved: C:\Users\vm-user\Downloads\nyiso_CAPITL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CENTRL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_MHK VL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_HUD VL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_LONGIL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_N.Y.C..csv
Saved: C:\Users\vm-user\Downloads\nyiso_WEST.csv
Saved: C:\Users\vm-user\Downloads\nyiso_GENESE.csv
Saved: C:\Users\vm-user\Downloads\nyiso_NORTH.csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD.csv
Saved: C:\Users\vm-user\Downloads\nyiso_MILLWD.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CENTRL_GENESE_WEST.csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD_HUD VL_N.Y.C..csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD_HUD VL_LONGIL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD_HUD VL_MILLWD.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CAPITL_MHK VL_NORTH.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CENTRL_GENESE_MHK VL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CAPITL_CENTRL_M